<a href="https://colab.research.google.com/github/amydst/Data-Science-24/blob/main/demographics_filtered_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.4'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("demographicsFilter").getOrCreate()

In [10]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/1/demographics.csv"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("demographics.csv"), inferSchema=True, sep=',')

# Show DataFrame
df.show()

+---+-----------------+---+--------+---------+--------+--------------------+---------------+------+--------------------+
| id|             name|age|height_m|weight_kg|children|          occupation|academic_degree|salary|            location|
+---+-----------------+---+--------+---------+--------+--------------------+---------------+------+--------------------+
|  1|    Glad Gavrieli| 38|    1.52|       74|       0|Computer Systems ...|       Bachelor|    78|           Louisiana|
|  2|  Henrieta Fittes| 34|    1.72|       39|       4|             Teacher|         Master|    44|            Illinois|
|  3|   Peyton Dulanty| 24|     1.8|       47|       5|Senior Quality En...|            PhD|    44|      North Carolina|
|  4|     Denna Morgen| 48|    1.81|       71|       5|   Account Executive|         Master|    81|          California|
|  5|    Camella Izaks| 34|    1.65|       60|       1|   Director of Sales|            PhD|    76|                Ohio|
|  6|     Shara Esposi| 49|     

In [11]:
# What occupation had the highest salary?

df.orderBy(df["salary"].desc()).show(1)


+---+-----------+---+--------+---------+--------+-----------------+---------------+------+--------+
| id|       name|age|height_m|weight_kg|children|       occupation|academic_degree|salary|location|
+---+-----------+---+--------+---------+--------+-----------------+---------------+------+--------+
|126|Dede Spritt| 26|    1.87|       78|       4|Chemical Engineer|            PhD|   120|  Nevada|
+---+-----------+---+--------+---------+--------+-----------------+---------------+------+--------+
only showing top 1 row



In [12]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- height_m: double (nullable = true)
 |-- weight_kg: integer (nullable = true)
 |-- children: integer (nullable = true)
 |-- occupation: string (nullable = true)
 |-- academic_degree: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- location: string (nullable = true)



In [14]:
# What occupation had the lowest salary?
df.orderBy(df["salary"]).select(['occupation','salary']).show(1)

+----------------+------+
|      occupation|salary|
+----------------+------+
|Product Engineer|    40|
+----------------+------+
only showing top 1 row



In [15]:
# What is the mean salary of this dataset?
from pyspark.sql.functions import avg
df.select(avg("salary")).show()

+-----------+
|avg(salary)|
+-----------+
|     79.475|
+-----------+



In [16]:
# What is the max and min of the Salary column?
from pyspark.sql.functions import max, min
df.select(max("salary"), min("salary")).show()

+-----------+-----------+
|max(salary)|min(salary)|
+-----------+-----------+
|        120|         40|
+-----------+-----------+



In [20]:
# Show all of the occupations where salaries were above 80k
from pyspark.sql.functions import count
df.filter('salary > 80').orderBy((df['salary'].desc())).select('occupation').show(10)


df.orderBy(df["salary"].desc()).show(1)

+--------------------+
|          occupation|
+--------------------+
|   Chemical Engineer|
| Assistant Professor|
|    Product Engineer|
| Electrical Engineer|
|    Junior Executive|
|        Food Chemist|
|  Biostatistician IV|
|     Statistician IV|
|Computer Systems ...|
|    Graphic Designer|
+--------------------+
only showing top 10 rows

+---+-----------+---+--------+---------+--------+-----------------+---------------+------+--------+
| id|       name|age|height_m|weight_kg|children|       occupation|academic_degree|salary|location|
+---+-----------+---+--------+---------+--------+-----------------+---------------+------+--------+
|126|Dede Spritt| 26|    1.87|       78|       4|Chemical Engineer|            PhD|   120|  Nevada|
+---+-----------+---+--------+---------+--------+-----------------+---------------+------+--------+
only showing top 1 row



In [ ]:
# BONUS
# What is the average age and height for each academic degree type?
# HINT: You will need to use `groupby` to solve this

avg_df = df.groupBy("academic_degree").avg()
avg_df.select("academic_degree", "avg(age)", "avg(height_meter)").show()

+---------------+------------------+------------------+
|academic_degree|          avg(age)|     avg(height_m)|
+---------------+------------------+------------------+
|            PhD| 42.87818696883853|1.7537393767705372|
|         Master|42.105095541401276|1.7606050955414014|
|       Bachelor| 43.85585585585586|1.7371771771771771|
+---------------+------------------+------------------+

